#### Conectando Python com API do Google Sheets

In [11]:
import pandas as pd
import gspread
import json
import os
import warnings
import unidecode
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
key_json = os.getenv('KEY_JSON')

# Converte a string JSON em um dicionário Python
google_client = gspread.service_account_from_dict(json.loads(key_json))

# Identificador único da planilha do Google Sheets
SHEET_ID = '1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ'

# Abre a planilha do Google Sheets usando o código único da planilha
spreadsheet = google_client.open_by_key(SHEET_ID)

# Seleciona a aba chamada 'Dados' dentro da planilha
worksheet = spreadsheet.worksheet('db')

# Obtém todos os valores da worksheet (retorna uma lista de listas)
dados = worksheet.get_all_values()

# A primeira linha é usada como o cabeçalho (nomes das colunas)
df = pd.DataFrame(dados[1:], columns=dados[0])

#### Limpeza e Preparação dos dados

In [12]:
# Padronizando o nome das colunas
df.rename(columns=str.lower, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

# Removendo os acentos das colunas
df.columns = [unidecode.unidecode(col) for col in df.columns]

In [13]:
# Converter colunas de data/hora para o tipo datetime
df['chegada_cd'] = pd.to_datetime(df['chegada_cd'])
df['inicio_do_carregamento'] = pd.to_datetime(df['inicio_do_carregamento'])
df['fim_do_carregamento'] = pd.to_datetime(df['fim_do_carregamento'])
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

# Lista de colunas que precisam ser convertidas para timedelta
timedelta_columns = [
    'tempo_de_espera', 
    'tempo_de_expedicao', 
    'tempo_total', 
    'hora_chegada_cd', 
    'hora_inicio_do_carregamento', 
    'hora_fim_do_carregamento'
]

# Aplicar a conversão para timedelta usando apply
df[timedelta_columns] = df[timedelta_columns].apply(pd.to_timedelta)

# Ajustar "quantidade_remessas" como tipo numérico (inteiro)
df['quantidade_remessas'] = pd.to_numeric(df['quantidade_remessas'], errors='coerce')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   unidade                      149 non-null    object         
 1   nome                         149 non-null    object         
 2   placa                        149 non-null    object         
 3   chegada_cd                   149 non-null    datetime64[ns] 
 4   inicio_do_carregamento       149 non-null    datetime64[ns] 
 5   fim_do_carregamento          149 non-null    datetime64[ns] 
 6   quantidade_remessas          149 non-null    int64          
 7   data                         149 non-null    datetime64[ns] 
 8   tempo_de_espera              149 non-null    timedelta64[ns]
 9   tempo_de_expedicao           149 non-null    timedelta64[ns]
 10  tempo_total                  149 non-null    timedelta64[ns]
 11  status_meta                  149

#### Transformando os dados com `groupby`

In [15]:
 # Tempo Médio de Expedição por Data
tempo_medio_dia = df.groupby(['data', 'unidade']).agg(
                    media_tempo_espera = pd.NamedAgg('tempo_de_espera', 'mean'),
                    media_tempo_expedicao = pd.NamedAgg('tempo_de_expedicao', 'mean'),
                    media_tempo_total = pd.NamedAgg('tempo_total', 'mean'),
                    total_remessas = pd.NamedAgg('quantidade_remessas', 'sum')).reset_index()

def format_timedelta(td):
    # Formata como HH:MM:SS
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Aplicar a formatação das médias
tempo_medio_dia[['media_tempo_espera',
                 'media_tempo_expedicao',
                 'media_tempo_total']] = tempo_medio_dia[['media_tempo_espera',
                                                         'media_tempo_expedicao',
                                                         'media_tempo_total']].applymap(format_timedelta)

tempo_medio_dia

,data,unidade,media_tempo_espera,media_tempo_expedicao,media_tempo_total,total_remessas
0,2024-10-15,HVGI - VILA GUILHERME,00:56:54,00:27:35,01:24:29,346
1,2024-10-16,HVGI - VILA GUILHERME,00:29:43,00:16:24,00:46:07,152
2,2024-10-18,HVGI - VILA GUILHERME,01:52:03,00:48:34,02:40:38,4277
3,2024-10-19,HVGI - VILA GUILHERME,03:33:29,00:15:34,03:49:03,1046
4,2024-10-21,HVGI - VILA GUILHERME,01:46:10,01:05:17,02:51:28,3001
